### Imports

In [2]:
import pickle
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda, Conv2D, MaxPool2D, Reshape, Conv2DTranspose
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
from keras import backend as K

### Load the training and testing data

In [3]:
with open('hw4_tr7.pkl', 'rb') as pickle_file:
    train_data = pickle.load(pickle_file)
print(train_data.shape)

(6265, 28, 28)


In [4]:
with open('hw4_te7.pkl', 'rb') as pickle_file:
    test_data = pickle.load(pickle_file)
print(test_data.shape)

(1028, 28, 28)


### Adding Channel Dimension

In [5]:
train_data = np.expand_dims(train_data, axis=3)

In [6]:
test_data = np.expand_dims(test_data, axis=3)

### Variational AutoEncoder

In [7]:
def VAE_model(ip_shape = [28, 28, 1], latent_dim = 4):
    input_img = Input(ip_shape) 
    
    encoder_model = Sequential()   
    encoder_model.add(Conv2D(16, kernel_size = (5, 5), input_shape = ip_shape, activation = 'relu'))
    encoder_model.add(Conv2D(32, kernel_size = (3, 3), activation = 'relu'))
    encoder_model.add(Conv2D(64, kernel_size = (2, 2), activation = 'relu'))
    encoder_model.add(Flatten())
    encoder_model.add(Dense(128))
    
    encoder_output = encoder_model(input_img)
    
    z_mean = Dense(latent_dim)(encoder_output)
    z_log_var = Dense(latent_dim)(encoder_output)
     
    epsilon =  K.random_normal(shape=K.shape(z_mean))
    #reparameterize
    z = z_mean + K.exp(0.5 + z_log_var) * epsilon
    
    decoder_model = Sequential()
    decoder_model.add(Dense(latent_dim*2, input_shape = z.get_shape(), activation = 'relu'))
    decoder_model.add(Dense(28*28, activation = 'relu'))
    decoder_model.add(Reshape(target_shape = (28,28, 1)))
    decoder_model.add(Conv2DTranspose(64, kernel_size = (2, 2), activation = 'relu', padding='SAME'))
    decoder_model.add(Conv2DTranspose(32, kernel_size = (3, 3), activation = 'relu', padding='SAME'))
    decoder_model.add(Conv2DTranspose(16, kernel_size = (5, 5), activation = 'relu', padding='SAME'))
    decoder_model.add(Conv2DTranspose(1, kernel_size = (5, 5), padding='SAME'))
    
    prediction = decoder_model(z)
      
    vae_model = Model(inputs = input_img, outputs = prediction)
    
    return vae_model, z_mean, z_log_var

### ELBO loss

In [8]:
def vae_loss(ip, op):
    kld = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    bce = tf.losses.sigmoid_cross_entropy(ip, op)
    
    return bce+kld

### Initialize and Compile Model

In [9]:
vae_model, z_mean, z_log_var = VAE_model() 
vae_model.compile(loss=vae_loss,optimizer = tf.keras.optimizers.Adam())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



## Training

In [ ]:
vae_model.fit(train_data, train_data, batch_size=16, epochs=50, shuffle = True, validation_data = (test_data, test_data))

Train on 6265 samples, validate on 1028 samples
Epoch 1/50
6265/6265 [==============================] - 48s 8ms/sample - loss: 0.2782 - val_loss: 0.2423
Epoch 2/50
6265/6265 [==============================] - 45s 7ms/sample - loss: 0.2435 - val_loss: 0.2398
Epoch 3/50
6265/6265 [==============================] - 45s 7ms/sample - loss: 0.2426 - val_loss: 0.2414
Epoch 4/50
6265/6265 [==============================] - 43s 7ms/sample - loss: 0.2422 - val_loss: 0.2392
Epoch 5/50
6265/6265 [==============================] - 44s 7ms/sample - loss: 0.2419 - val_loss: 0.2389
Epoch 6/50
6265/6265 [==============================] - 45s 7ms/sample - loss: 0.2419 - val_loss: 0.2388
Epoch 7/50
6265/6265 [==============================] - 44s 7ms/sample - loss: 0.2417 - val_loss: 0.2408
Epoch 8/50
6265/6265 [==============================] - 44s 7ms/sample - loss: 0.2414 - val_loss: 0.2388
Epoch 9/50
6265/6265 [==============================] - 45s 7ms/sample - loss: 0.2416 - val_loss: 0.2388
Epoch 1

In [ ]:
from keras.models import load_model

In [ ]:
vae_model.save('vae.h5')

In [ ]:
vae_model_2 = model.load('vae.h5')

## References
1. https://www.tensorflow.org/tutorials/generative/cvae
2. https://blog.keras.io/building-autoencoders-in-keras.html